5.Server and Client based TCP2
=======
---
### 1. Solution of echo_client
### 2. Application Protocol
### 3. I/O Buffer in TCP Socket
### 4. Stop and Wait Method
### 5. TCP Internal Operation Principle

---
#### 1. Solution of echo_client
5.1.1. Problem in echo_server and echo_client
In echo_server, the server have only one purpose that transmit the data received.
TCP is borderless protocol. Even the data is transmitted by dividing by two times of write() or three times of write(), there is no problem.

However, since client write and read the data in sentence by sentence, you have to distinguish the border of data. Terefore this method can be a problem. The funtions read() and write() of TCP doesn't distinguish the border of the data.

5.1.2. Soluton of echo_client
We need to insert the iterative code that read the length of transmitted data through calling the funtion write(). 
```c
#include <unistd.h>

ssize_t write(int fd, const void *buf, size_t count);
/* On success, the number bytes written is returned. On error, -1 is returned. */
ssize_t read(int fd, void *buf, size_t count);
/* On success, the number of bytes read is returned (zero indicates end of file). On error, -1 is returned and errno is set appropriately. */
```
######  Figure 5.1.1. The funtions write and read in UNIX
Following is the code that contains the code of classification required for reading data separately based on TCP.
```c
str_len = write(sock, message, strlen(message));
recv_len = 0;
while (recv_len < str_len) {
	recv_cnt = read(sock, &message[recv_len], BUF_SIZE-1 );
	if ( recv_cnt == -1 ) 
		error_handling("read() error!");
	recv_len += recv_cnt;
}
```
#### 2. Application Protocol
We  will implement a calculator program.
procedure)
1. Server get several numbers and operator information by client.
2. The server calculate based on the information and send the result to client.
	- Ex) information : 3, 5, 9, +
			 result : 3+5+9 = 17
	- EX) information : 4, 3, 2, -
			 result : 4-3-2 = -1

**< In UNIX BSD >**
[op_server.c][https://github.com/ndo04343/Learning_Basic-Network-Programming/blob/master/src/ch5/op_server.c](https://github.com/ndo04343/Learning_Basic-Network-Programming/blob/master/src/ch5/op_server.c)
[op_client.c][https://github.com/ndo04343/Learning_Basic-Network-Programming/blob/master/src/ch5/op_client.c](https://github.com/ndo04343/Learning_Basic-Network-Programming/blob/master/src/ch5/op_client.c)

#### 3. I/O Buffer in TCP Socket
- I/O buffers exist separately for each TCP socket.
- The I/O buffer is automatically created when creating a socket.
- Even if the socket is closed, the data remaining in **the output buffer continues to be transmitted**.
- But when the socket is closed, the data remaining in **the input buffer is destroyed**.

Since the socket exists, sliding window protocol of data can be used, so the buffers doesn't spill over.
#### 4. Stop and Wait Method
1. Flow control technique
	- 송신측과 수신측의 데이터 처리 속도 차이를 해결하기 위한 방법	
	- Stop-and-Wait 기법, Sliding Window 기법이 있음.
2. Stop-and wait 흐름 제어 기법
	- 전송 측이 프레임을 전송한 다음, 각 데이터 프레임에 대한 ACK를 기다림.
	- ACK 프레임이 도착하면 다음 프레임을 전송함
3. Sliding Window
	- 수신측에서 설정한 윈도우 크기만큼 송신측에서 확인 응답(ACK) 없이 패킷을 전송할 수 있어서 데이터 흐름을 동적으로 조절하는 기법
	- Window : 전송 및 수신 측에서 만들어진 버퍼의 크기
	- ACK 프레임이 도착하면, 전송측 윈도우는 ACK 프레임수에 따라 오른쪽 경계가 이동하여 윈도우 크기가 늘어남
    

#### 5. TCP Internal Operation Principle
1. Connection Setting
< TCP 3-way Handshake >
It refers to the process of establishing a session in advance with the other computer to ensure accurate transmission.
- SYN : synchronize sequence numbers
- ACK : acknowledgment

![enter image description here](https://t1.daumcdn.net/cfile/tistory/2701CF3D56E8FED503)

$$ ACK = 수신 SEQ 번호 + 1 $$

2. Data sending and receiving with relative socket
![enter image description here](https://t1.daumcdn.net/cfile/tistory/2476384656E8FED52F)
The reason of increase ACK value to size of bytes sent to relative socket is not also checking  whether the packet is transmitted or not but also whether data is lost.

$$ ACK -> SEQ + 전송된 바이트 크기 + 1 $$

 Also, If SEQ was sent, the timer operate. And if ACK about SEQ wasn't sent, the data is sent again.

3. Close with relative socket
![enter image description here](https://t1.daumcdn.net/cfile/tistory/230F094156E8FED534)
